In [1]:
import os 
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [2]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [3]:
vectorstore = Chroma(persist_directory="./new_test", embedding_function=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [15]:
### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
# qa_system_prompt = """You are an assistant for question-answering tasks. \
# Use the following pieces of retrieved context to answer the question. \
# If you don't know the answer, just say that you don't know. \
# Use three sentences maximum and keep the answer concise.\

# {context}"""
qa_system_prompt = """ You are a Artificial Intelligence Tutor of Signal ans System subject. \
            You task is to help student to understand Signal ans System subject concept \
            and help them to solve problem step by step. You should not solve the problem directly, \
            rather you help them to learn and solve the problem gradually and also make sure that \
            your response are in latex. If you cannot find the answer from the pieces of context, \
            just say that you don't know, don't try to make up an answer. \
            
            {context} """
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [16]:
result = conversational_rag_chain.invoke(
    {"input": "$y(t) = x(2t)$. Is it a linear system?"},
    config={"configurable": {"session_id": "abc123"}},
)

In [5]:
result = conversational_rag_chain.invoke(
    {"input": "What are the condition to saisfy a equation is linear system or not?"},
    config={"configurable": {"session_id": "abc123"}},
)

In [17]:
result

{'input': '$y(t) = x(2t)$. Is it a linear system?',
 'chat_history': [],
 'context': [Document(page_content='[\\hat{y}(t)=y(t-2)=x(t-4)+x(t)]', metadata={'source': '../mmd/Chapter_01.mmd'}),
  Document(page_content='[x(t)=a_{1}x_{1}(t)+a_{2}x_{2}(t)+\\cdot\\cdot\\cdot+a_{m}x_{m}(t)]\n\nthen, by virtue of linearity, the response (y(t)) is given by\n\n[y(t)=a_{1}y_{1}(t)+a_{2}y_{2}(t)+\\cdot\\cdot\\cdot+a_{m}y_{m}(t)]\n\nwhere (y_{k}(t)) is the zero-state response to an input (x_{k}(t)). This apparently trivial observation has profound implications. As we shall see repeatedly in later chapters, it proves extremely useful and opens new avenues for analyzing linear systems.', metadata={'source': '../mmd/Chapter_01.mmd'}),
  Document(page_content='is a linear system. The coefficients (a_{i}) and (b_{i}) in this equation can be constants or functions of time. Although here we proved only zero-state linearity, it can be shown that such systems are also zero-input linear and have the decomposi

In [18]:
result['answer']

"To determine if the system described by $y(t) = x(2t)$ is linear, we need to check if it satisfies the two properties of linearity: superposition and homogeneity.\n\n1. Superposition:\nLet's consider two input signals $x_1(t)$ and $x_2(t)$, and their corresponding outputs $y_1(t)$ and $y_2(t)$:\n$y_1(t) = x(2t)$ and $y_2(t) = x(2t)$\n\nNow, let's find the output for the combined input $x(t) = a_1x_1(t) + a_2x_2(t)$:\n$y(t) = x(2t) = a_1x_1(2t) + a_2x_2(2t)$\n\nIf we compare this with the definition of linearity, we see that the system does not satisfy the superposition property because the output is not equal to the sum of the individual outputs.\n\n2. Homogeneity:\nLet's consider an input signal $x(t)$ and its corresponding output $y(t)$:\n$y(t) = x(2t)$\n\nNow, let's find the output for the scaled input $ax(t)$:\n$y(t) = ax(2t)$\n\nIf we compare this with the definition of homogeneity, we see that the system does not satisfy the homogeneity property because the output is not equal t

In [19]:
from IPython.display import display, Latex, Markdown
display(Markdown(result['answer']))

To determine if the system described by $y(t) = x(2t)$ is linear, we need to check if it satisfies the two properties of linearity: superposition and homogeneity.

1. Superposition:
Let's consider two input signals $x_1(t)$ and $x_2(t)$, and their corresponding outputs $y_1(t)$ and $y_2(t)$:
$y_1(t) = x(2t)$ and $y_2(t) = x(2t)$

Now, let's find the output for the combined input $x(t) = a_1x_1(t) + a_2x_2(t)$:
$y(t) = x(2t) = a_1x_1(2t) + a_2x_2(2t)$

If we compare this with the definition of linearity, we see that the system does not satisfy the superposition property because the output is not equal to the sum of the individual outputs.

2. Homogeneity:
Let's consider an input signal $x(t)$ and its corresponding output $y(t)$:
$y(t) = x(2t)$

Now, let's find the output for the scaled input $ax(t)$:
$y(t) = ax(2t)$

If we compare this with the definition of homogeneity, we see that the system does not satisfy the homogeneity property because the output is not equal to the scaled version of the original output.

Therefore, the system described by $y(t) = x(2t)$ is not a linear system.

In [13]:
t = "The sinusoidal signal equation is given by $\( x(t) = \\sum_{n=1}^{\\infty} C_{n} \\cos(\\omega_{n}t + \\theta_{n}) \\)$.\nThe power of the signal can be calculated using the formula \\( P_{x} = \\frac{1}{2} \\sum_{n=1}^{\\infty} {C_{n}}^{2} \\).\nIf the signal also has a dc term, the power formula becomes \\( P_{x} = C_{0}^{2} + \\frac{1}{2} \\sum_{n=1}^{\\infty} {C_{n}}^{2} \\)."

In [14]:
from IPython.display import display, Latex, Markdown
display(Markdown(t))

The sinusoidal signal equation is given by $\( x(t) = \sum_{n=1}^{\infty} C_{n} \cos(\omega_{n}t + \theta_{n}) \)$.
The power of the signal can be calculated using the formula \( P_{x} = \frac{1}{2} \sum_{n=1}^{\infty} {C_{n}}^{2} \).
If the signal also has a dc term, the power formula becomes \( P_{x} = C_{0}^{2} + \frac{1}{2} \sum_{n=1}^{\infty} {C_{n}}^{2} \).